In [1]:
import time
import sys
import os
from os import listdir, makedirs
from os.path import isfile, join, isdir
import pickle
import gzip

import numpy as np
import torchfile
import matplotlib.pylab as plt
import matplotlib
import scipy.io as sio
from tqdm import tqdm


def getBBsize(points):
    mins = points.min(axis=0)
    maxs = points.max(axis=0)
    bbsize = maxs - mins
    normby = np.sqrt(bbsize[0] * bbsize[1])
    return np.sqrt(bbsize[0] * bbsize[1])


def calcDistance(pred, gt_pts):
    bbsize = getBBsize(gt_pts)
    return np.linalg.norm(pred - gt_pts, axis=1) / bbsize


def calculateMetrics(dists, error_np):
    errors = np.mean(dists, axis=0)
    np.save(error_np, errors)
    axes1 = np.linspace(0, 1, 1000)
    axes2 = np.zeros(1000)
    for i in range(1000):
        axes2[i] = (errors < axes1[i]).sum() / float(errors.shape[0])
    auc = np.sum(axes2[:70]) / 70
    return auc, np.mean(errors), np.std(errors)

In [2]:
out_pred_folder = 'out/'

db_eval = {
    "Menpo-3D": 'data/Menpo-3D/',
    '300W-Testset-3D': "data/300W-Testset-3D/",
    'AFLW2000-3D-Reannotated': "data/AFLW2000-3D-Reannotated/",
    'AFLW2000-3D': "data/AFLW2000/",
    '300VW-3D-CatA': "data/CatA/",
    '300VW-3D-CatB': "data/CatB/",
    '300VW-3D-CatC': "data/CatC/",
}

if not isdir('results/HGFPN'):
    makedirs('results/HGFPN')
aucs, mean_nmes, std_nmes = dict(), dict(), dict()
for dbkey, folder in db_eval.items():
    print("Dataset:", dbkey)
    is_mat = True if dbkey == 'AFLW2000-3D' else False 
    gt_folder = db_eval[dbkey]
    scores = None
    pred_folder = join(out_pred_folder, dbkey)
    suffix = ['.jpg', '.png']
    files = [
        os.path.join(dirpath, filename)
        for dirpath, dirnames, filenames in os.walk(gt_folder)
        for filename in filenames if filename[-4:] in suffix
    ]
    files.sort()
    try:
        with gzip.open(join(pred_folder, "all_pred.zip"), 'rb') as f:
            buffer = f.read()
        full_landmarks = pickle.loads(buffer)
    except Exception as e:
        print("Do not have file.")
        print("Exception: ", e)
        continue

    dists = None
    for nframe, imfile in tqdm(enumerate(files)):
        pts_file = imfile
        if is_mat:
            try:
                pts_file = imfile[:-4] + '.mat'
                gt = sio.loadmat(pts_file)['pt3d_68']
                groundtruth = gt.T[:,:2]
            except Exception as e:
                # print("Do not have groundtruth.")
                continue
        else:
            for file_type in suffix:
                pts_file = pts_file.replace(file_type, '.t7')
            try:
                groundtruth = torchfile.load(pts_file)
                landmarks = full_landmarks[imfile]
            except Exception as e:
                # print("Do not have groundtruth.")
                # print("Exception:", e)
                continue
        landmarks = full_landmarks[imfile]
    
        dist = calcDistance(landmarks, groundtruth)
        dist = dist.reshape((68, 1))
#         if np.mean(dist) < 0.01: # > 0.3
#             print(imfile)
        if dists is None:
            dists = dist
        else:
            dists = np.append(dists, dist, axis=1)
    scores = dists
    aucs[dbkey], mean_nmes[dbkey], std_nmes[dbkey] = calculateMetrics(dists, error_np='results/HGFPN/%s.npy' % dbkey)

0it [00:00, ?it/s]

Dataset: Menpo-3D


8955it [00:10, 874.17it/s]
262it [00:00, 2613.81it/s]

Dataset: 300W-Testset-3D


600it [00:00, 2478.05it/s]
232it [00:00, 2315.55it/s]

...

Do not have groundtruth.
Exception: [Errno 2] No such file or directory: 'data/CatC/559/1504.t7'
Do not have groundtruth.
Exception: [Errno 2] No such file or directory: 'data/CatC/559/1507.t7'


26463it [05:53, 88.72it/s]

Do not have groundtruth.
Exception: [Errno 2] No such file or directory: 'data/CatC/562/0125.t7'
Do not have groundtruth.
Exception: [Errno 2] No such file or directory: 'data/CatC/562/0126.t7'
Do not have groundtruth.
Exception: [Errno 2] No such file or directory: 'data/CatC/562/0127.t7'
Do not have groundtruth.
Exception: [Errno 2] No such file or directory: 'data/CatC/562/0128.t7'


27687it [06:08, 75.09it/s]


                  Dataset |  AUC  | Mean NME | StD NME
                 Menpo-3D | 71.97 |    1.96  |  1.72
          300W-Testset-3D | 78.89 |    1.43  |  0.42
  AFLW2000-3D-Reannotated | 74.22 |    1.86  |  2.46
              AFLW2000-3D | 62.47 |    2.71  |  2.59
                  300VW-A | 73.52 |    1.93  |  2.12
                  300VW-B | 74.63 |    1.94  |  2.70
                  300VW-C | 61.78 |    2.93  |  3.48


In [3]:
str_format = "%25s | %5.2f |   %5.2f  | %5.2f"
print("%25s |  AUC  | Mean NME | StD NME" % "Dataset")
for dbkey in db_eval.keys():
    print(str_format %(dbkey, aucs[dbkey] * 100, mean_nmes[dbkey] * 100, std_nmes[dbkey] * 100))

                  Dataset |  AUC  | Mean NME | StD NME
                 Menpo-3D | 71.97 |    1.96  |  1.72
          300W-Testset-3D | 78.89 |    1.43  |  0.42
  AFLW2000-3D-Reannotated | 74.22 |    1.86  |  2.46
              AFLW2000-3D | 62.47 |    2.71  |  2.59
                  300VW-A | 73.52 |    1.93  |  2.12
                  300VW-B | 74.63 |    1.94  |  2.70
                  300VW-C | 61.78 |    2.93  |  3.48
